In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
# Set the random seed for reproducibility
np.random.seed(42)

# Define the number of rows
num_rows = 10000

# Sample data generation
data = {
    'Customer_ID': range(1, num_rows + 1),
    'Company_Name': ['Company_' + str(i) for i in range(1, num_rows + 1)],
    'Industry': np.random.choice(['Technology', 'Healthcare', 'Education', 'Finance', 'Manufacturing'], num_rows),
    'Segment': np.random.choice(['Strategic', 'Enterprise', 'Mid-Market', 'Emerging'], num_rows, p=[0.1, 0.2, 0.4, 0.3]),
    'Company_Revenue': np.random.exponential(scale=200000, size=num_rows).astype(int),
    'ARR': np.random.exponential(scale=10000, size=num_rows).astype(int),
    'Contract_Value': np.random.exponential(scale=5000, size=num_rows).astype(int),
    'Product_Features_Used': np.random.poisson(5, num_rows),
    'Usage_Frequency': np.random.choice(['Daily', 'Weekly', 'Monthly'], num_rows),
    'Data_Usage': np.random.gamma(2, 2, num_rows).astype(int),
    'License_Count': np.random.poisson(10, num_rows),
    'Last_Activity_Date': pd.date_range(start='2021-01-01', periods=num_rows, freq='T').strftime('%Y-%m-%d')[np.random.choice(range(num_rows), num_rows)],
    'Engagement_Score': np.random.randint(1, 100, num_rows),
    'NPS_Score': np.random.randint(-100, 100, num_rows),
    'Support_Requests': np.random.poisson(2, num_rows),
    'Average_Response_Time': np.random.normal(12, 3.5, num_rows).astype(int),
    'Resolution_Time': np.random.normal(24, 5.5, num_rows).astype(int),
    'Health_Score': np.random.randint(1, 100, num_rows),
    'Churn_Risk_Score': np.random.randint(1, 100, num_rows),
    'Renewal_Date': pd.date_range(start='2024-01-01', periods=num_rows, freq='D').strftime('%Y-%m-%d')[np.random.choice(range(num_rows), num_rows)],
    'CSAT_Score': np.random.randint(1, 100, num_rows),
    'Customer_Feedback': np.random.choice(['Positive', 'Neutral', 'Negative'], num_rows),
    'Upsell_Opportunity': np.random.choice([True, False], num_rows, p=[0.2, 0.8]),
    'Referral_Count': np.random.poisson(1, num_rows),
    'Custom_Features': np.random.poisson(1, num_rows),
    'API_Calls': np.random.poisson(10, num_rows)
}

# Create DataFrame
df = pd.DataFrame(data)

# Adjustments for segment consistency
df.loc[df['Segment'] == 'Strategic', 'ARR'] *= 5
df.loc[df['Segment'] == 'Enterprise', 'ARR'] *= 3
df.loc[df['Segment'] == 'Mid-Market', 'ARR'] *= 2

df.loc[df['Segment'] == 'Strategic', 'Company_Revenue'] *= 5
df.loc[df['Segment'] == 'Enterprise', 'Company_Revenue'] *= 3
df.loc[df['Segment'] == 'Mid-Market', 'Company_Revenue'] *= 2

df.loc[df['Segment'] == 'Emerging', 'Health_Score'] = np.random.randint(20, 70, sum(df['Segment'] == 'Emerging'))
df.loc[df['Segment'] == 'Mid-Market', 'Health_Score'] = np.random.randint(40, 90, sum(df['Segment'] == 'Mid-Market'))
df.loc[df['Segment'] == 'Enterprise', 'Health_Score'] = np.random.randint(60, 100, sum(df['Segment'] == 'Enterprise'))
df.loc[df['Segment'] == 'Strategic', 'Health_Score'] = np.random.randint(80, 100, sum(df['Segment'] == 'Strategic'))



/var/folders/bf/8_s8z7kn3l979_b34wzrg6yr0000gn/T/ipykernel_29580/1081571586.py:20: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'Last_Activity_Date': pd.date_range(start='2021-01-01', periods=num_rows, freq='T').strftime('%Y-%m-%d')[np.random.choice(range(num_rows), num_rows)],


In [3]:

# Assign churn probabilities
churn_probs = {
    'Strategic': 0.1,  # 10% churn rate for Strategic
    'Enterprise': 0.15,  # 15% churn rate for Enterprise
    'Mid-Market': 0.18,  # 18% churn rate for Mid-Market
    'Emerging': 0.25   # 25% churn rate for Emerging
}

# Apply different churn rates based on the segment
df['Churned'] = np.random.rand(len(df)) < df['Segment'].map(churn_probs)



In [4]:
segment_csm_ranges = {
    'Strategic': (20, 50),
    'Enterprise': (50, 100),
    'Mid-Market': (100, 150),
    'Emerging': (100, 200)
}
def assign_csms(df, segment, ranges):
    customers = df[df['Segment'] == segment].index
    num_customers = len(customers)
    num_csms = np.random.randint(num_customers // ranges[1] + 1, num_customers // ranges[0] + 1)
    if num_csms == 0:
        num_csms = 1  # Ensure at least one CSM is assigned
    csm_ids = np.arange(1, num_csms + 1)
    csm_names = [f'CSM_{segment[:3]}_{id}' for id in csm_ids]
    assignment = np.random.choice(csm_names, num_customers)
    np.random.shuffle(assignment)  #patterned allocation avoidance
    df.loc[customers, 'CSM'] = assignment

# for each segment
for segment, ranges in segment_csm_ranges.items():
    assign_csms(df, segment, ranges)



In [5]:
# Reduce engagement metrics for churned customers
df.loc[df['Churned'] == 1, 'Engagement_Score'] *= 0.5
df.loc[df['Churned'] == 1, 'Health_Score'] *= 0.5

# Increase support issues for churned customers
df.loc[df['Churned'] == 1, 'Support_Requests'] += np.random.poisson(2, sum(df['Churned'] == 1))
df.loc[df['Churned'] == 1, 'Resolution_Time'] += np.random.normal(10, 2.5, sum(df['Churned'] == 1))

# Adjust renewal dates to past dates for churned customers
past_dates = pd.date_range(start='2020-01-01', periods=sum(df['Churned'] == 1), freq='D')
df.loc[df['Churned'] == 1, 'Renewal_Date'] = past_dates


/var/folders/bf/8_s8z7kn3l979_b34wzrg6yr0000gn/T/ipykernel_29580/1244776298.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 6.  34.5 13.  ... 26.  19.  19.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['Churned'] == 1, 'Engagement_Score'] *= 0.5
/var/folders/bf/8_s8z7kn3l979_b34wzrg6yr0000gn/T/ipykernel_29580/1244776298.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[23.  30.  40.5 ... 24.5 22.5 40.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['Churned'] == 1, 'Health_Score'] *= 0.5
/var/folders/bf/8_s8z7kn3l979_b34wzrg6yr0000gn/T/ipykernel_29580/1244776298.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[26.6559177

In [6]:
df['Last_Activity_Date'] = pd.to_datetime(df['Last_Activity_Date'])
df['Days_since_last_activity'] = (pd.Timestamp('2023-01-01') - df['Last_Activity_Date']).dt.days

# Add contract length based on renewal date and some starting date
df['Renewal_Date'] = pd.to_datetime(df['Renewal_Date'])
df['Contract_Start_Date'] = df['Renewal_Date'] - pd.to_timedelta(np.random.randint(180, 720, num_rows), unit='days')
df['Contract_length'] = (df['Renewal_Date'] - df['Contract_Start_Date']).dt.days


In [7]:

# Print the first few rows of the DataFrame to check

df.head()

,Customer_ID,Company_Name,Industry,Segment,Company_Revenue,ARR,Contract_Value,Product_Features_Used,Usage_Frequency,Data_Usage,License_Count,Last_Activity_Date,Engagement_Score,NPS_Score,Support_Requests,Average_Response_Time,Resolution_Time,Health_Score,Churn_Risk_Score,Renewal_Date,CSAT_Score,Customer_Feedback,Upsell_Opportunity,Referral_Count,Custom_Features,API_Calls,Churned,CSM,Days_since_last_activity,Contract_Start_Date,Contract_length
0,1,Company_1,Finance,Strategic,3250225,11625,8026,3,Daily,8,11,2021-01-07,46.0,-24,1,7,15.000000,89.0,40,2025-02-21,73,Negative,False,0,1,11,False,CSM_Str_7,724,2023-07-20,582
1,2,Company_2,Manufacturing,Emerging,56394,10024,4944,4,Daily,6,12,2021-01-04,6.0,-2,3,16,26.655918,23.0,42,2020-01-01,2,Neutral,False,2,0,10,True,CSM_Eme_17,727,2018-11-12,415
2,3,Company_3,Education,Mid-Market,3654,11088,553,0,Monthly,11,13,2021-01-02,34.5,-94,7,12,38.793407,30.0,94,2020-01-02,85,Positive,False,2,0,11,True,CSM_Mid_2,729,2019-03-23,285
3,4,Company_4,Manufacturing,Enterprise,341292,486,10925,2,Weekly,8,5,2021-01-03,81.0,-15,1,13,14.000000,84.0,53,2024-07-07,26,Positive,False,0,0,10,False,CSM_Ent_6,728,2023-11-26,224
4,5,Company_5,Manufacturing,Mid-Market,177870,570,12471,7,Weekly,3,9,2021-01-02,37.0,-47,0,14,17.000000,63.0,63,2042-03-31,68,Negative,False,1,2,9,False,CSM_Mid_7,729,2041-06-27,277


In [8]:
df.describe()

,Customer_ID,Company_Revenue,ARR,Contract_Value,Product_Features_Used,Data_Usage,License_Count,Last_Activity_Date,Engagement_Score,NPS_Score,Support_Requests,Average_Response_Time,Resolution_Time,Health_Score,Churn_Risk_Score,Renewal_Date,CSAT_Score,Referral_Count,Custom_Features,API_Calls,Days_since_last_activity,Contract_Start_Date,Contract_length
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000.00000
mean,5000.50000,4.535808e+05,21847.727000,5014.625800,4.996300,3.513500,10.022500,2021-01-03 23:32:55.680000,45.665250,0.028700,2.373500,11.535100,25.448990,58.178700,49.6851,2034-11-30 11:26:52.799999744,49.712100,1.003700,1.000400,9.980400,727.018800,2033-09-06 21:58:19.200000,449.56150
min,1.00000,9.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,2021-01-01 00:00:00,0.500000,-100.000000,0.000000,0.000000,4.000000,10.000000,1.0000,2020-01-01 00:00:00,1.000000,0.000000,0.000000,1.000000,724.000000,2018-02-03 00:00:00,180.00000
25%,2500.75000,1.039322e+05,5021.000000,1411.000000,3.000000,1.000000,8.000000,2021-01-02 00:00:00,21.000000,-50.000000,1.000000,9.000000,21.000000,40.000000,25.0000,2026-01-08 18:00:00,25.000000,0.000000,0.000000,8.000000,725.000000,2024-10-14 18:00:00,316.00000
50%,5000.50000,2.613980e+05,12779.500000,3431.000000,5.000000,3.000000,10.000000,2021-01-04 00:00:00,42.500000,-1.000000,2.000000,12.000000,25.000000,59.000000,50.0000,2034-08-14 12:00:00,50.000000,1.000000,1.000000,10.000000,727.000000,2033-06-27 12:00:00,449.00000
75%,7500.25000,5.809560e+05,27907.750000,7037.250000,6.000000,5.000000,12.000000,2021-01-06 00:00:00,69.000000,51.000000,3.000000,14.000000,29.867497,79.000000,74.0000,2043-02-26 06:00:00,74.000000,2.000000,2.000000,12.000000,729.000000,2041-12-02 00:00:00,585.00000
max,10000.00000,7.506540e+06,423270.000000,52442.000000,20.000000,24.000000,26.000000,2021-01-07 00:00:00,99.000000,99.000000,13.000000,25.000000,55.855712,99.000000,99.0000,2051-05-18 00:00:00,99.000000,7.000000,6.000000,27.000000,730.000000,2050-11-13 00:00:00,719.00000
std,2886.89568,5.813791e+05,27757.663243,5048.190415,2.237272,2.874969,3.184118,NaN,28.394441,58.318734,1.737153,3.516732,6.881964,23.034909,28.4170,NaN,28.515959,0.993572,1.002148,3.152271,1.987169,NaN,155.87167


In [9]:
df.to_csv('customer_data.csv', index=False)